In [34]:
# this script used Qualification stat to predict Final goals

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [3]:
df = pd.read_csv(r"D:\Semester 7\Machine Learning\Assignment\DataPoisson_wtemp_pop_gdp_stat.csv")
df.head()

,Unnamed: 0,home_team,away_team,home_goals,away_goals,Total_Score_home_team,Total_Score_away_team,HomeTeamRank_Difference,AwayTeamRank_Difference,Total_count_home_team,...,Hteam_gdp,Ateam_gdp,HT_Hattack,AT_Hattack,HT_Aattack,AT_Aattack,HT_Hdefense,AT_Hdefense,HT_Adefense,AT_Adefense
0,0,Spain,South Korea,2,2,58.0,0.0,5.8,-5.8,5,...,5.291220e+11,0.000000e+00,1.457249,1.934370,0.000000,1.432836,0.000000,0.278607,0.000000,0.248705
1,1,Colombia,Romania,1,3,51.3,55.3,7.4,-7.4,10,...,8.170350e+10,3.007444e+10,1.160622,2.368030,1.950249,2.055944,0.278607,0.685315,0.193437,0.546468
2,2,Belgium,Morocco,1,0,45.4,43.2,-3.9,3.9,3,...,2.461950e+11,3.560414e+10,0.728625,2.732342,2.569930,1.027972,0.000000,0.000000,0.273234,0.273234
3,3,Cameroon,Sweden,2,2,44.2,58.3,22.0,-22.0,7,...,1.060016e+10,2.263340e+11,1.160622,1.092937,0.626866,2.055944,0.557214,0.000000,0.290155,0.273234
4,4,Norway,Mexico,1,0,57.0,52.0,-9.1,9.1,4,...,1.271310e+11,5.278130e+11,3.278810,1.639405,1.541958,0.513986,0.513986,0.000000,0.273234,0.819703


In [4]:
df.columns

Index(['Unnamed: 0', 'home_team', 'away_team', 'home_goals', 'away_goals',
       'Total_Score_home_team', 'Total_Score_away_team',
       'HomeTeamRank_Difference', 'AwayTeamRank_Difference',
       'Total_count_home_team', 'Total_count_away_team', 'Country', 'Year',
       'tournament', 'host_temp', 'home_temp', 'away_temp', 'homeTemp_diff',
       'awayTemp_diff', 'home_pop', 'away_pop', 'Hteam_gdp', 'Ateam_gdp',
       'HT_Hattack', 'AT_Hattack', 'HT_Aattack', 'AT_Aattack', 'HT_Hdefense',
       'AT_Hdefense', 'HT_Adefense', 'AT_Adefense'],
      dtype='object')

In [5]:
#df.loc[df.tournament == 'FIFA World Cup']

In [25]:
#df2000 = df.loc[df.Year >= 2000].reset_index()

In [6]:
# split the dataset in to train and test
traindf, testdf = train_test_split(df, test_size=0.33, random_state=42)

In [7]:
goal_model_data = pd.concat([traindf[['home_team','away_team','home_goals','Total_Score_home_team',
                                       'Total_count_home_team','HomeTeamRank_Difference',
                                           'tournament','homeTemp_diff','home_pop','Hteam_gdp',
                                     'HT_Hattack','HT_Aattack','HT_Hdefense','HT_Adefense']].assign(home=1).rename(
            columns={'home_team':'team', 'away_team':'opponent','home_goals':'goals',
                     'Total_Score_home_team':'total_score','Total_count_home_team':'total_count',
                     'HomeTeamRank_Difference':'rank_difference','tournament':'tournament',
                    'homeTemp_diff':'Temp_diff','home_pop':'population','Hteam_gdp':'gdp',
                    'HT_Hattack':'Home_attackS','HT_Aattack':'Away_attackS','HT_Hdefense':'Home_defenseS',
                    'HT_Adefense':'Away_defenseS'}),
           traindf[['away_team','home_team','away_goals','Total_Score_away_team','Total_count_away_team',
                     'AwayTeamRank_Difference','tournament','awayTemp_diff','away_pop','Ateam_gdp',
                   'AT_Hattack','AT_Aattack','AT_Hdefense','AT_Adefense']].assign(home=0).rename(
            columns={'away_team':'team', 'home_team':'opponent','away_goals':'goals',
                    'Total_Score_away_team':'total_score','Total_count_away_team':'total_count',
                     'AwayTeamRank_Difference':'rank_difference','tournament':'tournament',
                    'awayTemp_diff':'Temp_diff','away_pop':'population','Ateam_gdp':'gdp',
                    'AT_Hattack':'Home_attackS','AT_Aattack':'Away_attackS','AT_Hdefense':'Home_defenseS',
                     'AT_Adefense':'Away_defenseS'})])

In [23]:
goal_model_data = pd.concat([traindf[['home_team','away_team','home_goals','Total_Score_home_team',
                                       'Total_count_home_team','HomeTeamRank_Difference',
                                           'tournament','homeTemp_diff','home_pop','Hteam_gdp',
                                     'HT_Hattack','HT_Hdefense']].assign(home=1).rename(
            columns={'home_team':'team', 'away_team':'opponent','home_goals':'goals',
                     'Total_Score_home_team':'total_score','Total_count_home_team':'total_count',
                     'HomeTeamRank_Difference':'rank_difference','tournament':'tournament',
                    'homeTemp_diff':'Temp_diff','home_pop':'population','Hteam_gdp':'gdp',
                    'HT_Hattack':'attackS','HT_Hdefense':'defenseS'}),
           traindf[['away_team','home_team','away_goals','Total_Score_away_team','Total_count_away_team',
                     'AwayTeamRank_Difference','tournament','awayTemp_diff','away_pop','Ateam_gdp',
                   'AT_Aattack','AT_Adefense']].assign(home=0).rename(
            columns={'away_team':'team', 'home_team':'opponent','away_goals':'goals',
                    'Total_Score_away_team':'total_score','Total_count_away_team':'total_count',
                     'AwayTeamRank_Difference':'rank_difference','tournament':'tournament',
                    'awayTemp_diff':'Temp_diff','away_pop':'population','Ateam_gdp':'gdp',
                    'AT_Aattack':'attackS','AT_Adefense':'defenseS'})])

In [24]:
goal_model_data.head()

,team,opponent,goals,total_score,total_count,rank_difference,tournament,Temp_diff,population,gdp,attackS,defenseS,home
104,Argentina,England,0,775.727273,4,-5.636364,FIFA World Cup,2.346417,20481779.0,9.772400e+10,1.055238,0.662151,1
274,Italy,Uruguay,0,1097.000000,10,3.333333,FIFA World Cup,-11.183125,50199700.0,2.151730e+12,1.589710,0.446296,1
118,South Africa,Spain,2,631.727273,5,28.181818,FIFA World Cup,5.132250,17099840.0,1.154820e+11,0.527619,0.000000,1
31,Nigeria,Italy,1,54.100000,4,4.800000,FIFA World Cup,17.548500,45138458.0,3.383304e+10,2.185874,0.000000,1
36,Sweden,Brazil,0,58.300000,5,3.900000,FIFA World Cup,-6.390417,7484656.0,2.263340e+11,1.092937,0.000000,1


In [33]:
#poisson_model= smf.glm(formula="goals ~rank_difference+Temp_diff+population+gdp+Home_attackS+Away_attackS+Home_defenseS+Away_defenseS", data=goal_model_data, 
#                        family=sm.families.Poisson()).fit()

#poisson_model= smf.glm(formula="goals ~rank_difference+Home_attackS+Away_attackS+Home_defenseS+Away_defenseS", data=goal_model_data, 
#                          family=sm.families.Poisson()).fit()
poisson_model= smf.glm(formula="goals ~rank_difference+population+Temp_diff+attackS", data=goal_model_data, 
                          family=sm.families.Poisson()).fit()

poisson_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  goals   No. Observations:                  474
Model:                            GLM   Df Residuals:                      469
Model Family:                 Poisson   Df Model:                            4
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -662.30
Date:                Wed, 18 Aug 2021   Deviance:                       523.55
Time:                        08:27:41   Pearson chi2:                     482.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.1964      0.086      2.282      0.022       0.028       0.365
rank_difference    -0.0134      0.002     -7.363      0.000      -0.017      -0.010
population      -9.596e-10   9.38e-10     -1.023      0.306    -2.8e-09    8.78e-10
Temp_diff           0.0038      0.004      1.084      0.278      -0.003       0.011
attackS             0.0014      0.046      0.030      0.976      -0.089       0.092
===================================================================================
"""

In [16]:
#def getPrediction(model,ht_rd,at_rd,ht_temp,at_temp,ht_pop,at_pop,ht_gdp,at_gdp):
def getPrediction(model,ht_rd,at_rd,ht_Hat,at_Hat,ht_Aat,at_Aat,ht_Hdef,at_Hdef,ht_Adef,at_Adef):
    predictdict = {}
    #home_team_goals = round(model.predict(pd.DataFrame(data = {'rank_difference': ht_rd,'Temp_diff':ht_temp,'population':ht_pop,'gdp':ht_gdp},index = [1])).values[0])
    #away_team_goals = round(model.predict(pd.DataFrame(data = {'rank_difference': at_rd,'Temp_diff':at_temp,'population':at_pop,'gdp':at_gdp},index = [1])).values[0])

    home_team_goals = round(model.predict(pd.DataFrame(data = {'rank_difference': ht_rd,'Home_attackS':ht_Hat,'Away_attackS':ht_Aat,'Home_defenseS':ht_Hdef,'Away_defenseS':ht_Adef},index = [1])).values[0])
    away_team_goals = round(model.predict(pd.DataFrame(data = {'rank_difference': at_rd,'Home_attackS':at_Hat,'Away_attackS':at_Aat,'Home_defenseS':at_Hdef,'Away_defenseS':at_Adef},index = [1])).values[0])

    if home_team_goals > away_team_goals:
        predictdict['pred'] = "HW"
    elif home_team_goals < away_team_goals:
        predictdict['pred'] = "AW"
    else:
        predictdict['pred'] = "D"
    return predictdict

In [17]:
testdf.columns

Index(['Unnamed: 0', 'home_team', 'away_team', 'home_goals', 'away_goals',
       'Total_Score_home_team', 'Total_Score_away_team',
       'HomeTeamRank_Difference', 'AwayTeamRank_Difference',
       'Total_count_home_team', 'Total_count_away_team', 'Country', 'Year',
       'tournament', 'host_temp', 'home_temp', 'away_temp', 'homeTemp_diff',
       'awayTemp_diff', 'home_pop', 'away_pop', 'Hteam_gdp', 'Ateam_gdp',
       'HT_Hattack', 'AT_Hattack', 'HT_Aattack', 'AT_Aattack', 'HT_Hdefense',
       'AT_Hdefense', 'HT_Adefense', 'AT_Adefense'],
      dtype='object')

In [18]:
wl_ = []
for index,row in testdf.iterrows():
    print(index)
    alldictpredict = {}
    
    home_team = row['home_team']
    away_team = row['away_team']
    ht_rd     = row['HomeTeamRank_Difference']
    at_rd     = row['AwayTeamRank_Difference']
    #ht_temp   = row['home_temp']
    #at_temp   = row['away_temp']
    #ht_pop    = row['home_pop']
    #at_pop    = row['away_pop']
    #ht_gdp    = row['Hteam_gdp']
    #at_gdp    = row['Ateam_gdp']
    ht_Hat    = row['HT_Hattack']
    at_Hat    = row['AT_Hattack']
    ht_Aat    = row['HT_Aattack']
    at_Aat    = row['AT_Aattack']
    ht_Hdef    = row['HT_Hdefense']
    at_Hdef    = row['AT_Hdefense']
    ht_Adef    = row['HT_Adefense']
    at_Adef    = row['AT_Adefense']
    #pred = getPrediction(poisson_model,ht_rd,at_rd,ht_temp,at_temp,ht_pop,at_pop,ht_gdp,at_gdp)
    pred = getPrediction(poisson_model,ht_rd,at_rd,ht_Hat,at_Hat,ht_Aat,at_Aat,ht_Hdef,at_Hdef,ht_Adef,at_Adef)
    alldictpredict['home_team'] = row['home_team']
    alldictpredict['away_team'] = row['away_team']
    #alldict['Goals'] = row['WonGoals']
    alldictpredict['pred'] = pred['pred']
    if row['home_goals']>row['away_goals']:
        alldictpredict['actual'] = "HW"
    elif row['home_goals']<row['away_goals']:
        alldictpredict['actual'] = "AW"
    else:
        alldictpredict['actual'] = "D"
    #alldictpredict['home_goals'] = row['home_goals']
    #alldictpredict['away_goals'] = row['away_goals']
    #print(alldict)
    wl_.append(alldictpredict)
    df_p_2000 = pd.DataFrame(wl_)
    
    #print(year,month,home_team,away_team) 

220
42
286
181
56
272
15
57
140
335
82
168
350
305
119
332
199
145
338
298
126
253
237
33
158
281
277
319
25
73
209
9
341
147
139
39
22
193
94
90
177
30
301
238
345
172
55
5
45
222
275
231
247
78
327
225
3
183
76
244
63
46
285
77
72
84
342
195
242
93
185
352
203
343
182
75
296
163
16
66
227
7
180
114
314
310
223
157
108
265
60
113
329
109
289
137
17
155
24
101
311
116
233
194
250
349
292
110
261
318
153
19
148
79
124
18
132


In [19]:
df_p_2000

,home_team,away_team,pred,actual
0,Ghana,Germany,AW,AW
1,Paraguay,Bulgaria,D,D
2,France,Nigeria,HW,HW
3,Spain,France,D,AW
4,South Africa,Denmark,D,D
...,...,...,...,...
112,Tunisia,Saudi Arabia,HW,D
113,Japan,Jamaica,D,AW
114,Belgium,Russia,D,HW
115,Switzerland,Colombia,D,AW


In [20]:
#y_pred = round(poisson_model.predict(testdf[['rank_difference','Temp_diff','population']]))

In [20]:
y_test = df_p_2000.actual
y_pred = df_p_2000.pred
conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix

array([[15, 17,  3],
       [ 6, 19,  5],
       [ 3, 30, 19]], dtype=int64)

In [21]:
df_confusion = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)
df_confusion = df_confusion.drop(['All'], axis = 1)
df_confusion = df_confusion.drop(['All'], axis = 0)
df_confusion.head()

Predicted,AW,D,HW
Actual,,,
AW,15,17,3
D,6,19,5
HW,3,30,19


In [22]:
#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred)))

print('Micro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, y_pred, average='weighted')))

from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_test, y_pred, target_names=['AW', 'D', 'HW']))


Accuracy: 0.45

Micro Precision: 0.45
Micro Recall: 0.45
Micro F1-score: 0.45

Macro Precision: 0.54
Macro Recall: 0.48
Macro F1-score: 0.46

Weighted Precision: 0.57
Weighted Recall: 0.45
Weighted F1-score: 0.47

Classification Report

              precision    recall  f1-score   support

          AW       0.62      0.43      0.51        35
           D       0.29      0.63      0.40        30
          HW       0.70      0.37      0.48        52

    accuracy                           0.45       117
   macro avg       0.54      0.48      0.46       117
weighted avg       0.57      0.45      0.47       117



In [26]:
# split the dataset in to train and test
traindf2000, testdf2000 = train_test_split(df2000, test_size=0.33, random_state=42)

In [42]:
df.columns

Index(['Unnamed: 0', 'home_team', 'away_team', 'home_goals', 'away_goals',
       'Total_Score_home_team', 'Total_Score_away_team',
       'HomeTeamRank_Difference', 'AwayTeamRank_Difference',
       'Total_count_home_team', 'Total_count_away_team', 'Country', 'Year',
       'tournament', 'host_temp', 'home_temp', 'away_temp', 'homeTemp_diff',
       'awayTemp_diff', 'home_pop', 'away_pop', 'Hteam_gdp', 'Ateam_gdp'],
      dtype='object')

In [43]:
goal_model_data_2000 = pd.concat([traindf2000[['home_team','away_team','home_goals','Total_Score_home_team',
                                       'Total_count_home_team','HomeTeamRank_Difference',''
                                           'tournament','homeTemp_diff','home_pop','Hteam_gdp','home_temp']].assign(home=1).rename(
            columns={'home_team':'team', 'away_team':'opponent','home_goals':'goals',
                     'Total_Score_home_team':'total_score','Total_count_home_team':'total_count',
                     'HomeTeamRank_Difference':'rank_difference','tournament':'tournament',
                    'homeTemp_diff':'Temp_diff','home_pop':'population','Hteam_gdp':'gdp','home_temp':'team_temp'}),
           traindf2000[['away_team','home_team','away_goals','Total_Score_away_team','Total_count_away_team',
                     'AwayTeamRank_Difference','tournament','awayTemp_diff','away_pop','Ateam_gdp','away_temp']].assign(home=0).rename(
            columns={'away_team':'team', 'home_team':'opponent','away_goals':'goals',
                    'Total_Score_away_team':'total_score','Total_count_away_team':'total_count',
                     'AwayTeamRank_Difference':'rank_difference','tournament':'tournament',
                    'awayTemp_diff':'Temp_diff','away_pop':'population','Ateam_gdp':'gdp','away_temp':'team_temp'})])

In [44]:
poisson_model_2000= smf.glm(formula="goals ~rank_difference+total_score+total_count+population+team_temp", data=goal_model_data_2000, 
                        family=sm.families.Poisson()).fit()
poisson_model_2000.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  goals   No. Observations:                16547
Model:                            GLM   Df Residuals:                    16541
Model Family:                 Poisson   Df Model:                            5
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -24057.
Date:                Tue, 17 Aug 2021   Deviance:                       21079.
Time:                        21:18:22   Pearson chi2:                 2.09e+04
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.0615      0.025      2.466      0.014       0.013       0.110
rank_difference    -0.0078      0.000    -62.325      0.000      -0.008      -0.008
total_score         0.0002   2.09e-05      7.189      0.000       0.000       0.000
total_count        -0.0054      0.002     -2.309      0.021      -0.010      -0.001
population       6.592e-10   8.53e-11      7.731      0.000    4.92e-10    8.26e-10
team_temp           0.0022      0.001      2.593      0.010       0.001       0.004
===================================================================================
"""

In [52]:
def getPrediction(model,ht_rd,at_rd,ht_pop,at_pop,ts_ht,ts_at,ht_tc,at_tc,ht_temp,at_temp):
    predictdict = {}
    home_team_goals = round(model.predict(pd.DataFrame(data = {'rank_difference': ht_rd,'total_score':ts_ht,'total_count':ht_tc,'population':ht_pop,'team_temp':ht_temp},index = [1])).values[0])
    away_team_goals = round(model.predict(pd.DataFrame(data = {'rank_difference': at_rd,'total_score':ts_at,'total_count':at_tc,'population':at_pop,'team_temp':at_temp},index = [1])).values[0])

    #if home_team_goals > away_team_goals:
    #    predictdict['pred'] = "HW"
    #elif home_team_goals < away_team_goals:
    #    predictdict['pred'] = "AW"
    #else:
    #    predictdict['pred'] = "D"


    if (home_team_goals > (away_team_goals+0.41)):
        predictdict['pred'] = "HW"
    elif (home_team_goals < (away_team_goals-0.41)):
        predictdict['pred'] = "AW"
    else:
        predictdict['pred'] = "D"

    return predictdict

In [53]:
wl_ = []
for index,row in testdf2000.iterrows():
    print(index)
    alldictpredict = {}
    
    home_team = row['home_team']
    away_team = row['away_team']
    ht_rd     = row['HomeTeamRank_Difference']
    at_rd     = row['AwayTeamRank_Difference']
    #ht_temp   = row['home_temp']
    #at_temp   = row['away_temp']
    ht_pop    = row['home_pop']
    at_pop    = row['away_pop']
    #ht_gdp    = row['Hteam_gdp']
    #at_gdp    = row['Ateam_gdp']
    ts_ht     = row['Total_Score_home_team']
    ts_at     = row['Total_Score_away_team']
    ht_tc     = row['Total_count_home_team']
    at_tc     = row['Total_count_home_team']
    ht_temp   = row['home_temp']
    at_temp   = row['away_temp']
    #pred = getPrediction(poisson_model_2000,ht_rd,at_rd,ht_temp,at_temp,ht_pop,at_pop,ht_gdp,at_gdp)
    pred = getPrediction(poisson_model_2000,ht_rd,at_rd,ht_pop,at_pop,ts_ht,ts_at,ht_tc,at_tc,ht_temp,at_temp)

    alldictpredict['home_team'] = row['home_team']
    alldictpredict['away_team'] = row['away_team']
    #alldict['Goals'] = row['WonGoals']
    alldictpredict['pred'] = pred['pred']
    if row['home_goals']>row['away_goals']:
        alldictpredict['actual'] = "HW"
    elif row['home_goals']<row['away_goals']:
        alldictpredict['actual'] = "AW"
    else:
        alldictpredict['actual'] = "D"
    #alldictpredict['home_goals'] = row['home_goals']
    #alldictpredict['away_goals'] = row['away_goals']
    #print(alldict)
    wl_.append(alldictpredict)
    df_p_2000_ = pd.DataFrame(wl_)
    
    #print(year,month,home_team,away_team) 

8626
1006
11999
9199
5863
8083
2150
2554
6259
9468
3295
218
10203
12223
385
4299
3455
5076
5396
8703
6580
3418
6532
8469
3649
8095
5595
247
6247
3693
11049
1795
2417
10605
11424
6762
8870
1457
3069
9035
10553
7798
9612
7534
12340
3746
6029
967
12030
4999
6298
3433
6697
8963
5858
12374
8131
2692
3366
7195
5493
2631
11950
10252
3721
11826
4176
5497
1984
4747
9141
429
4665
2254
8214
518
6965
11079
9960
6712
9408
6880
839
2021
8116
811
4490
4617
6735
2435
9699
4226
4015
8118
11631
8332
624
3617
2453
6703
11051
5491
6092
6608
1683
5720
8280
538
2238
3844
1579
4640
5835
9950
11904
6878
10756
6658
8715
5173
927
2301
3247
6606
8464
1068
3877
7607
11357
6415
10611
5781
7624
7741
200
8981
8650
5165
9258
4484
5226
12308
4112
3323
1078
10542
10109
993
8398
4414
311
7735
4680
12204
7932
6271
6372
9954
3872
7364
3529
5444
8685
5211
5725
8660
9718
6058
11280
4295
6909
10034
2344
10898
2865
3873
7928
1793
6427
11952
11891
3297
6625
7636
10068
8725
3143
10566
5958
2754
11737
2856
8027
5362
772
434
4688

5147
3834
5702
10711
7664
4850
7752
428
12013
3464
2412
6879
10379
4918
2392
2884
4338
10162
1061
4269
8317
11097
2619
2935
1432
5479
6921
704
12149
2250
2879
9968
9803
6888
608
2288
3621
6535
7811
2943
2144
3037
9159
2714
6657
4420
2034
758
11522
1864
2380
6454
12334
2109
3105
2055
399
930
10474
12135
542
11276
7153
4313
2085
10051
1418
4341
3585
2447
10820
9804
8748
1459
367
1393
2534
5826
5942
9353
4256
9992
9676
1670
6290
63
4509
8076
134
8081
11473
5239
3375
11143
11228
6208
7746
9964
11677
696
427
7116
1807
8974
3082
3379
12170
4513
4526
8480
4635
2910
6053
3299
676
1783
3222
4170
7559
11050
11736
6952
5324
6641
8109
6637
4601
6590
12051
11301
5466
6586
4894
9683
7307
7791
5376
4192
3181
1978
10434
8596
6128
2498
6069
740
0
4868
6520
10987
3100
12326
2340
4320
8097
6899
9522
6286
7337
12384
11171
8797
5245
10316
6835
11913
2677
5058
5264
10175
7495
7925
8657
8165
8810
8305
2791
1935
4342
12239
2901
216
5960
10507
10806
11806
921
866
10630
43
17
5746
6984
3616
8079
10375
11454
287

3860
1360
3963
6025
7401
486
8242
5814
3614
9214
9303
7990
3910
637
9149
11797
9563
11528
11984
3851
3378
11901
8071
12400
4028
10277
5666
167
1915
3411
7233
9063
6559
10848
8272
9939
801
2138
9270
5035
742
7375
6096
3470
10962
11961
3728
3884
11618
12195
9693
5894
2847
7315
2478
715
2957
10431
12330
3344
6755
9036
5562
4707
5070
2089
9749
9048
12273
8861
4398
1834
7550
6654
11380
12287
9924
10663
4049
3547
2485
3837
3755
12279
3592
1886
3003
23
10588
6836
1309
6214
5920
789
4672
5022
9890
8749
3219
9595
2514
11792
3753
8736
12388
8995
8293
6384
1957
3794
11871
10572
6400
12065
3487
6699
1395
9340
7759
5883
10094
12361
9927
6607
7008
3707
8170
9746
1226
6074
7819
12183
9571
5418
4030
10293
11047
10614
4308
3881
1310
4862
4012
5087
7578
10442
7257
10609
8411
1822
9411
1315
7191
4315
3785
12293
2315
2826
4023
7037
2595
12124
1339
3264
6459
738
7889
965
252
8752
5703
2098
5651
7072
8158
872
4031
5257
9023
4127
11703
7386
12074
11526
4439
4902
9864
9026
5990
11873
10913
10732
9473
10387
58

In [54]:
y_test = df_p_2000_.actual
y_pred = df_p_2000_.pred
conf_matrix = confusion_matrix(y_test, y_pred)
#conf_matrix

#importing accuracy_score, precision_score, recall_score, f1_score
#from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred)))

print('Micro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, y_pred, average='weighted')))

from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_test, y_pred, target_names=['AW', 'D', 'HW']))


Accuracy: 0.50

Micro Precision: 0.50
Micro Recall: 0.50
Micro F1-score: 0.50

Macro Precision: 0.53
Macro Recall: 0.50
Macro F1-score: 0.50

Weighted Precision: 0.59
Weighted Recall: 0.50
Weighted F1-score: 0.52

Classification Report

              precision    recall  f1-score   support

          AW       0.57      0.50      0.53      1175
           D       0.28      0.53      0.37       981
          HW       0.75      0.48      0.58      1948

    accuracy                           0.50      4104
   macro avg       0.53      0.50      0.50      4104
weighted avg       0.59      0.50      0.52      4104

